## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Clyde River,CA,70.4692,-68.5914,-1.97,84,0,0.00,clear sky
1,1,Bluff,NZ,-46.6000,168.3333,54.23,52,52,5.75,broken clouds
2,2,Ponta Do Sol,PT,32.6667,-17.1000,62.71,81,75,4.21,broken clouds
3,3,Belaya Gora,RU,68.5333,146.4167,-3.78,92,83,3.09,broken clouds
4,4,Bandarbeyla,SO,9.4942,50.8122,79.09,82,0,9.84,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Bandarbeyla,SO,9.4942,50.8122,79.09,82,0,9.84,clear sky
6,6,Bubaque,GW,11.2833,-15.8333,75.47,73,100,14.70,overcast clouds
8,8,Carnarvon,AU,-24.8667,113.6333,73.47,94,100,17.27,overcast clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,74.55,99,100,1.95,overcast clouds
11,11,Saint-Philippe,RE,-21.3585,55.7679,76.59,84,100,14.85,light rain
12,12,Vaini,TO,-21.2000,-175.2000,84.36,84,75,10.36,broken clouds
14,14,Hilo,US,19.7297,-155.0900,80.26,82,100,6.91,light rain
20,20,Rikitea,PF,-23.1203,-134.9692,76.93,70,99,16.82,overcast clouds
22,22,Airai,TL,-8.9266,125.4092,70.95,53,1,1.19,clear sky
24,24,Butaritari,KI,3.0707,172.7902,81.50,78,89,16.40,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Bandarbeyla,SO,9.4942,50.8122,79.09,82,0,9.84,clear sky
6,6,Bubaque,GW,11.2833,-15.8333,75.47,73,100,14.70,overcast clouds
8,8,Carnarvon,AU,-24.8667,113.6333,73.47,94,100,17.27,overcast clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,74.55,99,100,1.95,overcast clouds
11,11,Saint-Philippe,RE,-21.3585,55.7679,76.59,84,100,14.85,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bandarbeyla,SO,79.09,clear sky,9.4942,50.8122,
6,Bubaque,GW,75.47,overcast clouds,11.2833,-15.8333,
8,Carnarvon,AU,73.47,overcast clouds,-24.8667,113.6333,
9,Puerto Ayora,EC,74.55,overcast clouds,-0.7393,-90.3518,
11,Saint-Philippe,RE,76.59,light rain,-21.3585,55.7679,
12,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,
14,Hilo,US,80.26,light rain,19.7297,-155.0900,
20,Rikitea,PF,76.93,overcast clouds,-23.1203,-134.9692,
22,Airai,TL,70.95,clear sky,-8.9266,125.4092,
24,Butaritari,KI,81.50,light rain,3.0707,172.7902,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd></dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))